Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [49]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [50]:
import pandas as pd
import pandas_profiling

dtype_dict = {'ZIP CODE': 'object',
              'YEAR_BUILT': int}
# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv',
                 parse_dates=['SALE DATE'],
                 index_col='SALE DATE',
                 dtype=dtype_dict)  

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)
                 



In [51]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,,,,,,
2019-01-01,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0
2019-01-01,1,FASHION,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0
2019-01-01,1,FASHION,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0
2019-01-01,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014,1.0,0.0,1.0,0,500.0,0.0,2,R4,0
2019-01-01,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0


In [52]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [53]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23040 entries, 2019-01-01 to 2019-04-30
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  object 
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23039 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       23039 non-null  object 
 8   ADDRESS                         23040 non-null  object 
 9   APARTMENT_NUMBER                5201 non-null   object 
 10  ZIP_CODE                        23039 non-null  object 
 11  RESIDENTIAL_UNITS               23039 non-null  float64
 12  COMMERCIAL_UNIT

In [55]:
# Filtering the dataset 

# subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS'
filt1 = (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') 
df_subset = df.loc[filt1]

# the sale price was more than 100 thousand and less than 2 million.
filt2 = (df_subset['SALE_PRICE'] > 100000) & (df_subset['SALE_PRICE'] < 2000000)
df_subset = df_subset.loc[filt2]

print(df_subset.shape)
df_subset.head()


(3151, 20)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,,,,,,
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000
2019-01-01,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000
2019-01-02,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000


In [56]:
# we can drop whatever attributes we don't need from our subset 
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3151 entries, 2019-01-01 to 2019-04-30
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         3151 non-null   object 
 1   NEIGHBORHOOD                    3151 non-null   object 
 2   BUILDING_CLASS_CATEGORY         3151 non-null   object 
 3   TAX_CLASS_AT_PRESENT            3151 non-null   object 
 4   BLOCK                           3151 non-null   int64  
 5   LOT                             3151 non-null   int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       3151 non-null   object 
 8   ADDRESS                         3151 non-null   object 
 9   APARTMENT_NUMBER                1 non-null      object 
 10  ZIP_CODE                        3151 non-null   object 
 11  RESIDENTIAL_UNITS               3151 non-null   float64
 12  COMMERCIAL_UNITS

In [57]:

df_subset.BLOCK.nunique()

2496

In [58]:
# this could be used 
df_subset.LOT.nunique()

332

In [59]:
# this could be used 
df_subset.ZIP_CODE.nunique()

125

In [60]:
# this will add no value to our ML model
df_subset.TAX_CLASS_AT_TIME_OF_SALE.value_counts()

1    3151
Name: TAX_CLASS_AT_TIME_OF_SALE, dtype: int64

In [61]:
# Dropping the following attributes from our subset 
df_subset = df_subset.drop(['BUILDING_CLASS_CATEGORY', 'EASE-MENT', 'APARTMENT_NUMBER', 'ADDRESS', 'TAX_CLASS_AT_TIME_OF_SALE'], axis=1)
df_subset.head()

,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,
2019-01-01,3,OTHER,1,5495,801,A9,11230,1.0,0.0,1.0,"6,800",1325.0,1930.0,A9,550000
2019-01-01,4,OTHER,1,7918,72,A1,11427,1.0,0.0,1.0,"4,000",2001.0,1940.0,A1,200000
2019-01-02,2,OTHER,1,4210,19,A1,10461,1.0,0.0,1.0,"3,500",2043.0,1925.0,A1,810000
2019-01-02,3,OTHER,1,5212,69,A1,11226,1.0,0.0,1.0,"4,000",2680.0,1899.0,A1,125000
2019-01-02,3,OTHER,1,7930,121,A5,11203,1.0,0.0,1.0,"1,710",1872.0,1940.0,A5,620000


In [62]:
# remove comma from the LAND_SQUARE_FEET and converting into int
df_subset['LAND_SQUARE_FEET'] = df_subset['LAND_SQUARE_FEET'].str.replace(',', '').astype(int)
df_subset['LAND_SQUARE_FEET']

SALE DATE
2019-01-01    6800
2019-01-01    4000
2019-01-02    3500
2019-01-02    4000
2019-01-02    1710
              ... 
2019-04-30    3300
2019-04-30    2400
2019-04-30    4000
2019-04-30    2500
2019-04-30    1800
Name: LAND_SQUARE_FEET, Length: 3151, dtype: int64

In [63]:
# change YEAR_BUILT to int
df_subset['YEAR_BUILT'] = df_subset['YEAR_BUILT'].astype(int)

In [64]:
# convert zipcode to int
df_subset['ZIP_CODE'] = df_subset['ZIP_CODE'].astype(int)
df_subset['ZIP_CODE'].dtypes

dtype('int64')

In [65]:
import matplotlib.pyplot as plt



df_subset.hist(figsize=(12,12))
plt.show()

In [66]:
df_subset.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
BUILDING_CLASS_AT_PRESENT          object
ZIP_CODE                            int64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                    int64
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                          int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
dtype: object

# Time to split the data 

In [67]:
X = df_subset.drop('SALE_PRICE', axis=1)
y = df_subset['SALE_PRICE']

In [68]:
# Jan to march for train and April onwards to test

cutoff = '2019-04-01'
# creating a filt
filt = X.index < cutoff
# applying the filter 
X_train, y_train = X.loc[filt], y.loc[filt]

X_test, y_test = X.loc[~filt], y.loc[~filt]

In [69]:
# Insanity check 
X_train.shape[0] + X_test.shape[0] == 3151

True

# Time to make the Baseline prediction 

In [70]:
baseline_pred = y_train.mean()
baseline_pred

621573.7423214999

In [71]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print('Mean Absolute Error:', mean_absolute_error(y_train, [y_train.mean()]*len(y_train)))
print('Mean Squared Error:', mean_squared_error(y_train, [y_train.mean()]*len(y_train), squared=False))

Mean Absolute Error: 214721.52773001452
Mean Squared Error: 291549.06127392803


# Using OHE for our cat variables

In [74]:
from category_encoders import OneHotEncoder

# ohe = OneHotEncoder(cols=['NEIGHBORHOOD', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'BOROUGH', 'TAX_CLASS_AT_PRESENT'],use_cat_names=True)

# # Transform the training data 
# ohe_train = ohe.fit_transform(X_train)

# # transform the test data
# ohe_test = ohe.transform(X_test)

# ohe_train.head()

# Scaling the data 

In [75]:
from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# # scaling the train data
# scaled_train = scaler.fit_transform(ohe_train)
# # scaling the test data
# scaled_test = scaler.transform(ohe_test)

# Pipelines

###making num and cat pipeline 

In [76]:
from sklearn.pipeline import Pipeline, make_pipeline
# seperating categorical columns and numerical columns
# categorical attributes
df_subset_cat = df_subset.select_dtypes(include='object')
df_subset_cat.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'TAX_CLASS_AT_PRESENT',
       'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [77]:
# numerical attributes 
df_subset_num = df_subset.select_dtypes(exclude='object').drop('SALE_PRICE', axis=1)
df_subset_num.columns

Index(['BLOCK', 'LOT', 'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS',
       'TOTAL_UNITS', 'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT'],
      dtype='object')

In [78]:
# cat pipeline 
cat_pipeline = Pipeline([['ohe', OneHotEncoder(cols=['NEIGHBORHOOD', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'BOROUGH', 'TAX_CLASS_AT_PRESENT'],use_cat_names=True)],
                          ['scaler', StandardScaler()]])

# num pipeline
num_pipeline = Pipeline([['scaler', StandardScaler()]])

### Putting num and cat pipeline into one final pipeline 

In [79]:
# using column Transformer putting them together 
from sklearn.compose import ColumnTransformer

# getting the list of columns 
num_attribs = list(df_subset_num)
cat_attribs = list(df_subset_cat)

final_pipeline = ColumnTransformer([('numerical', num_pipeline, num_attribs), ('categorical', cat_pipeline, cat_attribs)])

# fit transform the final pipeline to the train set 
X_train_final = final_pipeline.fit_transform(X_train)
# trasnform pipeline to the test set
X_test_final = final_pipeline.transform(X_test)

# Pick the best model 

### Ridge Model

In [80]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# finding the best value of alpha 
ridge = Ridge(random_state=42)
parameters = {'alpha':[1e-15, 1e-10, 1e-8, 1e-5, 1e-2, 1, 5,10, 20,40, 50, 60, 100, 110]}

ridge_reg = RandomizedSearchCV(ridge, parameters, cv=10, scoring= 'neg_mean_squared_error', random_state=42)
ridge_reg.fit(X_train_final, y_train)

print(ridge_reg.best_params_)
print(ridge_reg.best_estimator_)
print(np.sqrt(-ridge_reg.best_score_))

{'alpha': 20}
Ridge(alpha=20, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=42, solver='auto', tol=0.001)
218178.45637918625


In [81]:
# using the best estimator we will predict the sale price 

ridge = ridge_reg.best_estimator_

sale_price_pred = ridge.predict(X_train_final)


print('Mean Absolute Error:', mean_absolute_error(y_train, sale_price_pred))
print('Mean Squared Error:', mean_squared_error(y_train, sale_price_pred, squared=False))
print('r2 score:', r2_score(y_train, sale_price_pred))

Mean Absolute Error: 150444.50040901007
Mean Squared Error: 212890.33040522234
r2 score: 0.4668019196879334


In [82]:
# Scores on the test set

sale_price_pred_test = ridge.predict(X_test_final)


print('Mean Absolute Error:', mean_absolute_error(y_test, sale_price_pred_test))
print('Mean Squared Error:', mean_squared_error(y_test, sale_price_pred_test, squared=False))
print('r2 score:', r2_score(y_test, sale_price_pred_test))

Mean Absolute Error: 155670.9580098343
Mean Squared Error: 225338.18847294143
r2 score: 0.42337931183521516


### Combining Ridge and Polynomial features 

In [83]:
from sklearn.preprocessing import PolynomialFeatures

poly_reg = make_pipeline(PolynomialFeatures(degree=5), Ridge(alpha= 20))
# fit on the transformed data
poly_reg.fit(X_train_final, y_train)
# predict sale price
sale_price_pred_poly = poly_reg.predict(X_train_final)


print('Mean Absolute Error:', mean_absolute_error(y_train, sale_price_pred_poly))
print('Mean Squared Error:', mean_squared_error(y_train, sale_price_pred_poly, squared=False))
print('r2 score:', r2_score(y_train, sale_price_pred_poly))

In [84]:
# Scores on the test set

sale_price_pred_poly_test = poly_reg.predict(X_test_final)


print('Mean Absolute Error:', mean_absolute_error(y_test, sale_price_pred_poly_test))
print('Mean Squared Error:', mean_squared_error(y_test, sale_price_pred_poly_test, squared=False))
print('r2 score:', r2_score(y_test, sale_price_pred_poly_test))

### Combinng Ridge and SGDRegressor 

In [85]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(random_state=42, penalty='l2', max_iter=100000)
# lets start by finding out the best value of the learing rate 

parameters1 = {'eta0': [0.0001, 0.001, 0.005, 0.01, 0.03, 0.06, 0.09, 1, 1.05]}

sgd_regressor = RandomizedSearchCV(sgd_reg, parameters1, cv=10, n_iter=10,  scoring = 'neg_mean_squared_error', random_state=42)
sgd_regressor.fit(X_train_final,y_train)

print(sgd_regressor.best_params_)
print(sgd_regressor.best_estimator_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


{'eta0': 0.0001}
SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.0001, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=100000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=42,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)


In [86]:
# Training dataset
sgd_reg = sgd_regressor.best_estimator_

sale_price_pred_sgd = sgd_reg.predict(X_train_final)

print('Mean Absolute Error:', mean_absolute_error(y_train, sale_price_pred_sgd))
print('Mean Squared Error:', mean_squared_error(y_train, sale_price_pred_sgd, squared=False))
print('r2 score:', r2_score(y_train, sale_price_pred_sgd))

Mean Absolute Error: 150486.18280366587
Mean Squared Error: 212877.1869095857
r2 score: 0.46686775518449986


In [87]:
# test dataset
sale_price_pred_sgd_test = sgd_reg.predict(X_test_final)


print('Mean Absolute Error:', mean_absolute_error(y_test, sale_price_pred_sgd_test))
print('Mean Squared Error:', mean_squared_error(y_test, sale_price_pred_sgd_test, squared=False))
print('r2 score:', r2_score(y_test, sale_price_pred_sgd_test))

Mean Absolute Error: 155806.55732675747
Mean Squared Error: 225390.065790356
r2 score: 0.4231137822504186


# Elastic net and SGDRegressor

In [88]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(random_state=42, penalty='elasticnet', l1_ratio= 0.3, max_iter=100000)
# lets start by finding out the best value of the learing rate 

parameters1 = {'eta0': [0.0005, 0.0001, 0.001, 0.005, 0.01, 0.03, 0.06, 0.09, 1, 1.05]}

sgd_regressor = RandomizedSearchCV(sgd_reg, parameters1, cv=9, n_iter=10,  scoring = 'neg_mean_squared_error', random_state=42)
sgd_regressor.fit(X_train_final,y_train)

print(sgd_regressor.best_params_)
print(sgd_regressor.best_estimator_)

{'eta0': 0.0001}
SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.0001, fit_intercept=True, l1_ratio=0.3,
             learning_rate='invscaling', loss='squared_loss', max_iter=100000,
             n_iter_no_change=5, penalty='elasticnet', power_t=0.25,
             random_state=42, shuffle=True, tol=0.001, validation_fraction=0.1,
             verbose=0, warm_start=False)


In [89]:
sgd_reg = sgd_regressor.best_estimator_

sale_price_pred_sgd1 = sgd_reg.predict(X_train_final)

print('Mean Absolute Error:', mean_absolute_error(y_train, sale_price_pred_sgd1))
print('Mean Squared Error:', mean_squared_error(y_train, sale_price_pred_sgd1, squared=False))
print('r2 score:', r2_score(y_train, sale_price_pred_sgd1))

Mean Absolute Error: 150485.99326135803
Mean Squared Error: 212876.96181854827
r2 score: 0.46686888262542836


In [90]:
sale_price_pred_sgd_test1 = sgd_reg.predict(X_test_final)


print('Mean Absolute Error:', mean_absolute_error(y_test, sale_price_pred_sgd_test1))
print('Mean Squared Error:', mean_squared_error(y_test, sale_price_pred_sgd_test1, squared=False))
print('r2 score:', r2_score(y_test, sale_price_pred_sgd_test1))

Mean Absolute Error: 155807.2794760244
Mean Squared Error: 225389.92212968427
r2 score: 0.42311451764960173
